In [29]:
import pandas as pd
import numpy as np
import json
import random
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [2]:
df = pd.read_csv(
    "data/transformed/df.csv",
    dtype={
        "product_title": str,
        "review_title": str,
        "review_text": str,
        "product_id": str,
    },
)
reviews_scraped = pd.read_csv(
    "data/transformed/reviews_scraped.csv",
    dtype={"review_title": str, "review_text": str, "product_id": str},
)

In [30]:
i = 0
ms = []
for _, row in df.iterrows():
    m = dict()
    if (
        np.isnan(row["rating_avg"])
        or np.isnan(row["rating_count"])
        or np.isnan(row["rating1"])
        or np.isnan(row["rating2"])
        or np.isnan(row["rating3"])
        or np.isnan(row["rating4"])
        or np.isnan(row["rating5"])
    ):
        continue
    pr_id = row["product_id"]
    m["product_id"] = pr_id
    m["review_title"] = row["review_title"]
    m["review_text"] = (
        row["review_text"]
        if len(row["review_text"]) < 500
        else row["review_text"][0:500] + "..."
    )
    m["category"] = row["PRODUCT_CATEGORY"]
    m["rating"] = row["rating"]
    m["verified"] = row["verified"]
    m["rating_avg"] = row["rating_avg"]
    m["rating_count"] = row["rating_count"]
    m["rating1"] = row["rating1"]
    m["rating2"] = row["rating2"]
    m["rating3"] = row["rating3"]
    m["rating4"] = row["rating4"]
    m["rating5"] = row["rating5"]
    m["label"] = row["label"]
    m["prediction"] = row["label"] if random.random() > 0.2 else row["label"] * -1 + 1
    i += 1
    ms.append(m)
    others = []
    # find all matching reviews:
    for _, r in reviews_scraped[reviews_scraped["product_id"] == pr_id].iterrows():
        if type(r["review_text"]) == float:
            continue
        mm = dict()
        mm["review_title"] = r["review_title"]
        # print(r["review_text"])
        mm["review_text"] = (
            r["review_text"]
            if len(r["review_text"]) < 500
            else r["review_text"][0:500] + "..."
        )
        if not np.isnan(r["helpful"]):
            mm["helpful"] = r["helpful"]
        mm["verified"] = r["verified"]
        mm["rating"] = r["rating"]
        others.append(mm)
    m["other_reviews"] = others
    if i > 300:
        break


f = open("game/public/data.json", "w+")
f.write(json.dumps(ms))
f.close()